In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

files = [
    'https://github.com/sambiner/LMU-Datathon-2024/blob/main/EPA%20Contract%20Data/FY2018_068_Contracts_Full_20240214_1.csv?raw=true',
    'https://github.com/sambiner/LMU-Datathon-2024/blob/main/EPA%20Contract%20Data/FY2019_068_Contracts_Full_20240214_1.csv?raw=true',
    'https://github.com/sambiner/LMU-Datathon-2024/blob/main/EPA%20Contract%20Data/FY2020_068_Contracts_Full_20240214_1.csv?raw=true',
    'https://github.com/sambiner/LMU-Datathon-2024/blob/main/EPA%20Contract%20Data/FY2020_068_Contracts_Full_20240214_1.csv?raw=true',
    'https://github.com/sambiner/LMU-Datathon-2024/blob/main/EPA%20Contract%20Data/FY2021_068_Contracts_Full_20240214_1.csv?raw=true',
    'https://github.com/sambiner/LMU-Datathon-2024/blob/main/EPA%20Contract%20Data/FY2022_068_Contracts_Full_20240214_1.csv?raw=true',
    'https://github.com/sambiner/LMU-Datathon-2024/blob/main/EPA%20Contract%20Data/FY2023_068_Contracts_Full_20240214_1.csv?raw=true',
    'https://github.com/sambiner/LMU-Datathon-2024/blob/main/EPA%20Contract%20Data/FY2024_068_Contracts_Full_20240214_1.csv?raw=true'
]

# Load the data from CSV files
df_list = [pd.read_csv(file, low_memory=False) for file in files]
df = pd.concat(df_list, ignore_index=True)

# Ensure 'action_date_fiscal_year' is of integer type
df['action_date_fiscal_year'] = pd.to_numeric(df['action_date_fiscal_year'], errors='coerce', downcast='integer')

nan_counts = df[['epa_designated_product', 'place_of_manufacture', 'recovered_materials_sustainability_code']].isna().sum(axis=1)

# Remove rows where at least two of the specified columns have NaN values
df = df[nan_counts < 2]
# Create a new 'sustainability_factor' column as a combination of the three sustainability columns.
df['sustainability_factor'] = (
    df['epa_designated_product'].astype(str) + "_" +
    df['place_of_manufacture'].astype(str) + "_" +
    df['recovered_materials_sustainability_code'].astype(str)
)

# Group by 'action_date_fiscal_year' and 'sustainability_factor', then calculate the mean number of offers received
trend_data = df.groupby(['action_date_fiscal_year', 'sustainability_factor'])['number_of_offers_received'].mean().reset_index()

# Create a rank column within each 'recovered_materials_sustainability_code' group based on the mean number of offers received
trend_data['rank'] = trend_data.groupby('action_date_fiscal_year')['number_of_offers_received'].rank(method='first', ascending=False)

# Filter to keep only top 2 for each 'recovered_materials_sustainability_code' category per year
top_factors_each_year = trend_data[trend_data['rank'] <= 2]

# Pivot the data for plotting
top_factors_pivot = top_factors_each_year.pivot(index='action_date_fiscal_year', columns='sustainability_factor', values='number_of_offers_received')


# Plotting the lineplot with the top 2 ranks for each year
plt.figure(figsize=(14, 7))
sns.lineplot(data=top_factors_pivot, dashes=False, markers=True, linewidth=2)
plt.title('Top 2 Sustainability Factors by Average Number of Offers Received (2018-2024)')
plt.xlabel('Fiscal Year')
plt.ylabel('Average Number of Offers Received')
plt.xticks([2018, 2019, 2020, 2021, 2022, 2023, 2024])  # Explicitly setting the x-axis ticks
plt.legend(title='Sustainability Factor', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()


ModuleNotFoundError: No module named 'seaborn'

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

files = [
    'https://github.com/sambiner/LMU-Datathon-2024/blob/main/EPA%20Contract%20Data/FY2018_068_Contracts_Full_20240214_1.csv?raw=true',
    'https://github.com/sambiner/LMU-Datathon-2024/blob/main/EPA%20Contract%20Data/FY2019_068_Contracts_Full_20240214_1.csv?raw=true',
    'https://github.com/sambiner/LMU-Datathon-2024/blob/main/EPA%20Contract%20Data/FY2020_068_Contracts_Full_20240214_1.csv?raw=true',
    'https://github.com/sambiner/LMU-Datathon-2024/blob/main/EPA%20Contract%20Data/FY2020_068_Contracts_Full_20240214_1.csv?raw=true',
    'https://github.com/sambiner/LMU-Datathon-2024/blob/main/EPA%20Contract%20Data/FY2021_068_Contracts_Full_20240214_1.csv?raw=true',
    'https://github.com/sambiner/LMU-Datathon-2024/blob/main/EPA%20Contract%20Data/FY2022_068_Contracts_Full_20240214_1.csv?raw=true',
    'https://github.com/sambiner/LMU-Datathon-2024/blob/main/EPA%20Contract%20Data/FY2023_068_Contracts_Full_20240214_1.csv?raw=true',
    'https://github.com/sambiner/LMU-Datathon-2024/blob/main/EPA%20Contract%20Data/FY2024_068_Contracts_Full_20240214_1.csv?raw=true'
]

# Load the data from CSV files
df_list = [pd.read_csv(file, low_memory=False) for file in files]
df = pd.concat(df_list, ignore_index=True)

# Clean data
# Remove rows with NaN values in the columns of interest
df = df.dropna(subset=['action_date_fiscal_year', 'parent_award_type', 'current_total_value_of_award'])

# Make sure 'action_date_fiscal_year' is the correct data type
# df['action_date_fiscal_year'] = pd.to_datetime(df['action_date']).dt.year

# Filter rows to include only the years 2018 - 2024
df = df[df['action_date_fiscal_year'].between(2018, 2024)]

# Convert 'current_total_value_of_award' to numeric, errors='coerce' will replace non-numeric values with NaN
df['current_total_value_of_award'] = pd.to_numeric(df['current_total_value_of_award'], errors='coerce')

# Drop any rows that could not be converted to numeric values in 'current_total_value_of_award'
df = df.dropna(subset=['current_total_value_of_award'])

# Group the data by year and parent_award_type
grouped_data = df.groupby(['action_date_fiscal_year', 'parent_award_type'])['current_total_value_of_award'].sum().reset_index()

# Now, plot the grouped bar chart
plt.figure(figsize=(15, 7))
sns.barplot(
    data=grouped_data,
    x='action_date_fiscal_year',
    y='current_total_value_of_award',
    hue='parent_award_type'
)
plt.title('Total Value of Awards by Parent Award Type (2018-2024)')
plt.xlabel('Fiscal Year')
plt.ylabel('Total Value of Award ($)')
plt.legend(title='Parent Award Type', bbox_to_anchor=(1.02, 1), loc='upper left')
plt.tight_layout()
plt.show()


ModuleNotFoundError: No module named 'seaborn'

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

files = [
    'https://github.com/sambiner/LMU-Datathon-2024/blob/main/EPA%20Contract%20Data/FY2018_068_Contracts_Full_20240214_1.csv?raw=true',
    'https://github.com/sambiner/LMU-Datathon-2024/blob/main/EPA%20Contract%20Data/FY2019_068_Contracts_Full_20240214_1.csv?raw=true',
    'https://github.com/sambiner/LMU-Datathon-2024/blob/main/EPA%20Contract%20Data/FY2020_068_Contracts_Full_20240214_1.csv?raw=true',
    'https://github.com/sambiner/LMU-Datathon-2024/blob/main/EPA%20Contract%20Data/FY2020_068_Contracts_Full_20240214_1.csv?raw=true',
    'https://github.com/sambiner/LMU-Datathon-2024/blob/main/EPA%20Contract%20Data/FY2021_068_Contracts_Full_20240214_1.csv?raw=true',
    'https://github.com/sambiner/LMU-Datathon-2024/blob/main/EPA%20Contract%20Data/FY2022_068_Contracts_Full_20240214_1.csv?raw=true',
    'https://github.com/sambiner/LMU-Datathon-2024/blob/main/EPA%20Contract%20Data/FY2023_068_Contracts_Full_20240214_1.csv?raw=true',
    'https://github.com/sambiner/LMU-Datathon-2024/blob/main/EPA%20Contract%20Data/FY2024_068_Contracts_Full_20240214_1.csv?raw=true'
]

# Load the data from CSV files
df_list = [pd.read_csv(file, low_memory=False) for file in files]
df = pd.concat(df_list, ignore_index=True)

# Clean data
# Remove rows with NaN values in the columns of interest
df = df.dropna(subset=['action_date', 'recovered_materials_sustainability', 'current_total_value_of_award'])

# Convert 'action_date_fiscal_year' to the correct data type
df['action_date_fiscal_year'] = pd.to_datetime(df['action_date'], errors='coerce').dt.year

# Filter rows to include only the years 2018 - 2024
df = df[df['action_date_fiscal_year'].between(2018, 2024)]

# Convert 'current_total_value_of_award' to numeric, errors='coerce' will replace non-numeric values with NaN
df['current_total_value_of_award'] = pd.to_numeric(df['current_total_value_of_award'], errors='coerce')

# Drop any rows that could not be converted to numeric values in 'current_total_value_of_award'
df = df.dropna(subset=['current_total_value_of_award'])

# Group the data by year and recovered_materials_sustainability
grouped_data = df.groupby(['action_date_fiscal_year', 'recovered_materials_sustainability'])['current_total_value_of_award'].sum().reset_index()

# Now, plot the grouped bar chart
plt.figure(figsize=(15, 7))
sns.barplot(
    data=grouped_data,
    x='action_date_fiscal_year',
    y='current_total_value_of_award',
    hue='recovered_materials_sustainability'
)
plt.title('Total Value of Awards by Recovered Materials Sustainability (2018-2024)')
plt.xlabel('Fiscal Year')
plt.ylabel('Total Value of Award ($)')
plt.legend(title='Recovered Materials Sustainability', bbox_to_anchor=(1.02, 1), loc='upper left')
plt.tight_layout()
plt.show()
